# Finetune a DistilRoBERTa model on the ELI5 Dataset

Task Description: Masked Language Modelling (MLM) predicts a masked token in a sequence. If we give it a sequence where some tokens inside are masked, it can predict the masked tokens.

Original Tutorial: https://huggingface.co/docs/transformers/tasks/masked_language_modeling

In [ ]:
!pip install -q transformers datasets evaluate accelerate

# Load ELI5 dataset

In [ ]:
from datasets import load_dataset

eli5 = load_dataset("eli5", split = "train_asks[:5000]")

In [ ]:
# Split the dataset into a train and test set
eli5 = eli5.train_test_split(test_size=0.2)

In [ ]:
# Look at the data
import pprint
pprint.pprint(eli5['train'][0])

# The text column is our model input


In [ ]:
# Preprocessing
## Load Model
from transformers import AutoTokenizer

checkpoint = "distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Preprocessing
We need to create a preprocess function that we will apply to every instance in the dataset. The preprocess function needs to:

1. Flatten the instance so that the text column is easily accessible
2. Join any list of strings
3. Tokenize result

Some token sequences will be **longer** than the maximum input length for the model. Hence we use a second preprocessing function to:

1. concatenate all token sequences
2. Split the concatenated sequences into shorter chunks defined by a `block_size` parameter.

In [ ]:
# The text field is nested so we need to flatten each instance
eli5 = eli5.flatten()
pprint.pprint(eli5['train'][0])

In [ ]:
# Operation to apply to every instance
print(" ".join(eli5['train']['answers.text'][0]), "\n")
print(tokenizer(" ".join(eli5['train']['answers.text'][0])) )

In [ ]:
# Wrap in a preprocess function
def preprocess_function(examples):
  return tokenizer([" ".join(x) for x in examples["answers.text"]])

In [ ]:
# Apply preprocessing over entire dataset - batched = True process multiple elements of the datasets
tokenized_eli5 = eli5.map(preprocess_function, batched = True, num_proc=4, remove_columns=eli5['train'].column_names)

In [ ]:
tokenized_eli5['train'][0]

In [ ]:
def group_texts(examples, block_size: int = 128):
  # This function is to cut the length of the text examples

  # Concatencate all texts
  concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
  total_length = len(concatenated_examples[list(examples.keys())[0]])

  if total_length >= block_size:
    total_length = (total_length // block_size) * block_size
  # Split by chunks of block size
  result = {
      k: [t[i: i + block_size] for i in range(0, total_length, block_size)]
      for k, t in concatenated_examples.items()
  }
  result["labels"] = result["input_ids"].copy()
  return result


In [ ]:
# Apply second preprocessing over entire dataset
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

In [ ]:
pprint.pprint(lm_dataset['train'][0])

In [ ]:
# Create a batch of examples, with dynamic padding. Use the appropriate collator function
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm_probability = 0.15)

# Train using the Trainer API
The main training steps are:

1. Define training hyperparameters using a model specific TrainingArguments function. At the end of each epoch, the Trainer will evaluate the defined loss metric and save the training checkpoint.

2. Pass the training arguments to a Trainer function alongside the model, dataset, tokenizer, data collator.

3. Call train() to finetune the model

In [ ]:
from transformers import AutoModelForMaskedLM, TrainingArguments, Trainer

model = AutoModelForMaskedLM.from_pretrained(checkpoint)

In [ ]:
training_args = TrainingArguments(
    output_dir = "eli5_mlm",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    fp16=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

In [ ]:
# Evaluate the fine tuned model and obtain the perplexity score
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.save_model("eli5_masked_model")

In [ ]:
# In this case, the tokenizer was not saved automatically, save it manually in the model folder for inference
tokenizer.save_pretrained("eli5_masked_model", legacy_format=False)

# Inference

Use model for inference using a pipeline wrapper

In [ ]:
text = "The Milky Way is a <mask> galaxy"

In [ ]:
from transformers import pipeline

mask_filler = pipeline("fill-mask", model = "eli5_masked_model")
mask_filler(text, top_k = 3)

In [ ]:
# Inference Pipeline using Pytorch
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("eli5_masked_model")
inputs = tokenizer(text, return_tensors="pt").input_ids
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]

model = AutoModelForMaskedLM.from_pretrained("eli5_masked_model")
logits = model(**inputs).logits
mask_token_logits = logits[0, mask_token_index, :]

top_3_tokens = torch.topk(mask_token_logits, 3, dim=1).indices[0].tolist()

for token in top_3_tokens:
  print(text.replace(tokenizer.mask_token, tokenizer.decode([token])))

In [ ]:
# Decode the generated token ids back into text
tokenizer.batch_decode(outputs, skip_special_tokens=True)